In [1]:
import pandas as pd

In [19]:
all_genes_df = pd.read_table('GRCh38_annotation.tsv', header=0, sep='\t', low_memory=False)
all_genes_df['p_value'] = 1
all_genes_df['fold_change'] = 0
all_genes_df['gene_symbol'] = all_genes_df['external_gene_name'].apply(lambda x: x.upper())
all_genes_df = all_genes_df.drop_duplicates(['gene_symbol'])
all_genes_df = all_genes_df.set_index('gene_symbol')
print(all_genes_df.shape)
all_genes_df.head()


(56630, 6)


,gene_id,entrezgene_id,external_gene_name,description,p_value,fold_change
gene_symbol,,,,,,
TSPAN6,ENSG00000000003,7105,TSPAN6,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858],1,0
TNMD,ENSG00000000005,64102,TNMD,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757],1,0
DPM1,ENSG00000000419,8813,DPM1,dolichyl-phosphate mannosyltransferase subunit...,1,0
SCYL3,ENSG00000000457,57147,SCYL3,SCY1 like pseudokinase 3 [Source:HGNC Symbol;A...,1,0
C1ORF112,ENSG00000000460,55732,C1orf112,chromosome 1 open reading frame 112 [Source:HG...,1,0


In [13]:
def get_upload_df(all_genes_df, concept_filename, p_value=0.01, fold_change=2):
    
    concept_df = pd.read_table(concept_filename, header=None, names=['gene_symbol'])
    concept_df['gene_symbol'] = concept_df['gene_symbol'].apply(lambda x: x.upper())
    concept_df = concept_df.set_index('gene_symbol')
    concept_df['concept'] = concept_filename
    #concept_df.head()

    upload_df = all_genes_df.join(concept_df, how='outer')
    upload_df.loc[upload_df['concept']==concept_filename, ['p_value', 'fold_change']] = (p_value, fold_change)
    #upload_df.head()
    
    display(upload_df.loc[:, ['p_value', 'fold_change', 'concept']].apply(pd.Series.value_counts))
    
    return upload_df


In [23]:
input_files = [
    'GO_SKELETAL_MUSCLE_CONTRACTION', 
    'HALLMARK_E2F_TARGETS',
    'HALLMARK_TNFA_SIGNALING_VIA_NFKB',
    'mm_KEGG_PPAR_SIGNALING_PATHWAY',
    ]

for file in input_files:
    upload_df = get_upload_df(all_genes_df, file +'.txt')
    upload_df.to_csv(file+'-upload.tsv', sep='\t', index=True)


,p_value,fold_change,concept
0.0,NaN,56587.0,NaN
0.01,43.0,NaN,NaN
1.0,56587.0,NaN,NaN
2.0,NaN,43.0,NaN
GO_SKELETAL_MUSCLE_CONTRACTION.txt,NaN,NaN,43.0


,p_value,fold_change,concept
0.0,NaN,56432.0,NaN
0.01,200.0,NaN,NaN
1.0,56432.0,NaN,NaN
2.0,NaN,200.0,NaN
HALLMARK_E2F_TARGETS.txt,NaN,NaN,200.0


,p_value,fold_change,concept
0.0,NaN,56430.0,NaN
0.01,200.0,NaN,NaN
1.0,56430.0,NaN,NaN
2.0,NaN,200.0,NaN
HALLMARK_TNFA_SIGNALING_VIA_NFKB.txt,NaN,NaN,200.0


,p_value,fold_change,concept
0.0,NaN,56567.0,NaN
0.01,76.0,NaN,NaN
1.0,56567.0,NaN,NaN
2.0,NaN,76.0,NaN
mm_KEGG_PPAR_SIGNALING_PATHWAY.txt,NaN,NaN,76.0
